In [10]:
#Importing dependencies
import re
import pandas as pd
import os

#These are characters we are looking for to find the curvenames
#before each section is a tilda
COMM = '~'
#~C is where the curvenames start
START = 'C'
#~A is where the data begins
END = 'A'
#AD1 and AD2 are characters in the line after ~C but before the curvenames
AD1 = "#"
AD2 = "-"

#Curvenames are stored in a dictionary with key = curvename, & value = unit

#RAW curvenames
#- Thickness curves, - nominal curves, & - Metal loss curves
MTDE_RAW = {'DEPT': 'FT', 'CNOIS': 'mV', 'IS': 'mV', 'COFS': 'mV', 'SR00': 'mV', 'SR01': 'mV', 'SR02': 'mV', 'SR03': 'mV',
      'SR04': 'mV', 'SR05': 'mV', 'SR06': 'mV', 'SR07': 'mV', 'SR08': 'mV', 'SR09': 'mV', 'SR10': 'mV', 'SR11': 'mV',
      'ANOIS': 'mV', 'IL': 'mV', 'AOFS': 'mV', 'LR00': 'mV', 'LR01': 'mV', 'LR02': 'mV', 'LR03': 'mV', 'LR04': 'mV',
      'LR05': 'mV', 'LR06': 'mV', 'LR07': 'mV', 'LR08': 'mV', 'LR09': 'mV', 'LR10': 'mV', 'LR11': 'mV', 'LR12': 'mV',
      'LR13': 'mV', 'LR14': 'mV', 'LR15': 'mV', 'LR16': 'mV', 'LR17': 'mV', 'LR18': 'mV', 'LR19': 'mV', 'LR20': 'mV',
      'LR21': 'mV', 'LR22': 'mV', 'LR23': 'mV', 'LR24': 'mV', 'LR25': 'mV', 'LR26': 'mV', 'LR27': 'mV', 'LR28': 'mV',
      'LR29': 'mV', 'LR30': 'mV', 'LR31': 'mV', 'LR32': 'mV', 'LR33': 'mV', 'LR34': 'mV', 'LR35': 'mV', 'LR36': 'mV',
      'LR37': 'mV', 'LR38': 'mV', 'LR39': 'mV', 'LR40': 'mV', 'LR41': 'mV', 'LR42': 'mV', 'LR43': 'mV', 'LR44': 'mV',
      'LR45': 'mV', 'LR46': 'mV', 'LR47': 'mV', 'LR48': 'mV', 'LR49': 'mV', 'LR50': 'mV', 'LR51': 'mV', 'LR52': 'mV',
      'LR53': 'mV', 'LR54': 'mV', 'LR55': 'mV', 'LR56': 'mV', 'LR57': 'mV', 'LR58': 'mV', 'LR59': 'mV', 'TEMP': 'degF',
      'GR': 'gAPI', 'LSPD': 'FT/M', 'TENS': 'lb'}

#Processes curvenames
#W/ 4" 7" and 9" tubing curves
MTDE_PROC_4_7_9 = {"DEPT" : "FT", "GR" : "gAPI", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches",
             "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown",
             "SN04" : "Unknown","SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown","LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}
#W/ 4" 7" 9" and 13" tubing curves
MTDE_PROC_4_7_9_13 = {"DEPT" : "FT", "GR" : "gAPI", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "THK13" : "inches", "ML13" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches",
             "NTH13" : "inches", "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown",
             "SN04" : "Unknown","SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown","LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}

MTDE_PROC_4_7_9_13_18 = {"DEPT" : "FT", "GR" : "gAPI", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "THK13" : "inches", "ML13" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches",
             "NTH13" : "inches","NTH18" : "inches", "THK18" : "inches","ML18" : "%",
             "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown",
             "SN04" : "Unknown","SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown","LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}

#W/ 2" 4" 4"TBG 7" and 9" tubing curves
MTDE_PROC_2_4_4tbg_7_9 = {"DEPT" : "FT", "GR" : "gAPI", "THK2" : "inches","ML2" : "%", "THK4-TBG" : "inches","ML4-TBG" : "%", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches", "NTH4-TBG" :  "inches","NTH2" :  "inches",
             "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown","SN04" : "Unknown",
             "SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown","LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}

#W/ 4" 4"LNR 7" and 9" tubing curves
MTDE_PROC_4_4lnr_7_9 = {"DEPT" : "FT", "GR" : "gAPI", "THK4-LNR" : "inches","ML4-LNR" : "%", "THK4" : "inches","ML4" : "%","THK7" : "inches", "ML7" : "%","THK9" : "inches",
             "ML9" : "%", "NTH4" : "inches","NTH7" : "inches","NTH9" :  "inches", "NTH4-LNR" :  "inches",
             "THK13" : "inches", "ML13" : "%", "NTH13" : "inches",
             "SN00" : "Unknown","SN01" : "Unknown","SN02" : "Unknown","SN03" : "Unknown","SN04" : "Unknown",
             "SN05" : "Unknown","SN06" : "Unknown","SN07" : "Unknown","SN08" : "Unknown",
             "SN09" : "Unknown","SN10" : "Unknown","SN11" : "Unknown","LN00" : "Unknown","LN01" : "Unknown",
             "LN02" : "Unknown","LN03" : "Unknown","LN04" : "Unknown","LN05" : "Unknown","LN06" : "Unknown",
             "LN07" : "Unknown","LN08" : "Unknown","LN09" : "Unknown","LN10" : "Unknown","LN11" : "Unknown",
             "LN12" : "Unknown","LN13" : "Unknown","LN14" : "Unknown","LN15" : "Unknown","LN16" : "Unknown",
             "LN17" : "Unknown","LN18" : "Unknown","LN19" : "Unknown","LN20" : "Unknown","LN21" : "Unknown",
             "LN22" : "Unknown","LN23" : "Unknown","LN24" : "Unknown","LN25" : "Unknown","LN26" : "Unknown",
             "LN27" : "Unknown","LN28" : "Unknown","LN29" : "Unknown","LN30" : "Unknown","LN31" : "Unknown",
             "LN32" : "Unknown","LN33" : "Unknown","LN34" : "Unknown","LN35" : "Unknown","LN36" : "Unknown",
             "LN37" : "Unknown","LN38" : "Unknown","LN39" : "Unknown","LN40" : "Unknown","LN41" : "Unknown",
             "LN42" : "Unknown","LN43" : "Unknown","LN44" : "Unknown","LN45" : "Unknown","LN46" : "Unknown",
             "LN47" : "Unknown","LN48" : "Unknown","LN49" : "Unknown","LN50" : "Unknown","LN51" : "Unknown",
             "LN52" : "Unknown","LN53" : "Unknown","LN54" : "Unknown","LN55" : "Unknown","LN56" : "Unknown",
             "LN57" : "Unknown","LN58" : "Unknown","LN59" : "Unknown","TNTH" : "inches", "TTHK" : "inches", "TML": "%"}

#We can add other dictionaries based on each job


#Gathers the headers
headers = []

CURVE_START = False
AFTER_DESC = False
FIN = False

#Name of the file that we are inputting. We can add a longer file path if the file is not inside the same folder
filename = "MP PROCESS.las"

# Inputting data from las file into variable td
with open(filename, 'r') as td:
    
    #going through line by line
    for line in td:
        #Checks for the end of the curvenames
        if (line[0] == COMM) and (line[1] == END):
            #Breaks out of the for loop
            break
        #Once the curvenames start we can change the boolean to allow for the next if statement to enter
        elif (line[0] == COMM) and (line[1] == START):
            CURVE_START = True

        if CURVE_START:
            #If the line we are reading is the line right after the ~C line, ignore it and continue
            if (line[0] == AD1) and (line[1] == AD2):
                #Changes the boolean to allow the next if statement to enter
                AFTER_DESC = True
                continue
            
            #Allow for the reading of the curvenames since we passed the ~C line and the description line
            if AFTER_DESC:
                #.strip removes any leading and trailing spaces. ex: "  banana  ".strip() = "banana"
                currline = line.strip()
                #Split the line across the character ":", we only care about the left side of the split
                headers.append(currline.split(":"))
        
    

curves = {}
#Loop over the items taken from the file
for x in headers:
    #Strip to remove any unneeded spaces
    curr = x[0].strip()
    #Split it across the period to go from NAME.UNIT to Name & Unit in two variables
    name, unit = curr.split(".")
    #Add those to a dictionary
    curves[name] = unit

#This print will provide you feedback on what the dictionary now looks like
print(curves)

{'DEPT': 'FT', 'GR': 'gAPI', 'THK4': 'inches', 'ML4': '%', 'THK7': 'inches', 'ML7': '%', 'THK9': 'inches', 'ML9': '%', 'THK13': 'inches', 'ML13': '%', 'NTH4': 'inches', 'NTH7': 'inches', 'NTH9': 'inches', 'NTH13': 'inches', 'SN00': 'Unknown', 'SN01': 'Unknown', 'SN02': 'Unknown', 'SN03': 'Unknown', 'SN04': 'Unknown', 'SN05': 'Unknown', 'SN06': 'Unknown', 'SN07': 'Unknown', 'SN08': 'Unknown', 'SN09': 'Unknown', 'SN10': 'Unknown', 'SN11': 'Unknown', 'LN00': 'Unknown', 'LN01': 'Unknown', 'LN02': 'Unknown', 'LN03': 'Unknown', 'LN04': 'Unknown', 'LN05': 'Unknown', 'LN06': 'Unknown', 'LN07': 'Unknown', 'LN08': 'Unknown', 'LN09': 'Unknown', 'LN10': 'Unknown', 'LN11': 'Unknown', 'LN12': 'Unknown', 'LN13': 'Unknown', 'LN14': 'Unknown', 'LN15': 'Unknown', 'LN16': 'Unknown', 'LN17': 'Unknown', 'LN18': 'Unknown', 'LN19': 'Unknown', 'LN20': 'Unknown', 'LN21': 'Unknown', 'LN22': 'Unknown', 'LN23': 'Unknown', 'LN24': 'Unknown', 'LN25': 'Unknown', 'LN26': 'Unknown', 'LN27': 'Unknown', 'LN28': 'Unknown

In [13]:
#Dictionary that has the values that have been checked
checked = {}

#The dictionary that is referenced. This is from the first part of the code and should be written based on the well diagram
dict_ref = MTDE_PROC_4_7_9_13

#Counting the number of curves we have checked
totalcurves = 0

#loop over the curves from the input file
for x in curves:
    #if they are in the reference dictionary
    if (x in dict_ref):
        #& if the unit is correct
        if(curves[x] == dict_ref[x]):
            #Add it to the checked dictionary
            checked[x] = curves[x]
            #Add 1 to the totalcurve count
            totalcurves+=1
        #If the unit is incorrect it will notify you with a print    
        else:
            print("issue with the unit for", x , curves[x])
    #If the curvename is not in the reference dictionary it will notify you below
    else:
        checked[x] = curves[x]
        print("CURVE ", x, " IS NOT IN ",  filename)

#If the curvenames are all correct it will give you this result        
if dict_ref == checked:
    print("All curvenames and units are correct")
    print(totalcurves, " Curves are in the las file ", filename)
#Else run through the checks below to find the issue    
else:
    #If the length of the reference dictionary is less than those checked means that there might be some extra curves in the input file
    if(len(dict_ref) < len(checked)):
        print("there might be extra curves")
    #If the length of the reference dictionary is greater than those checked means that there might be some missing curves in the input file
    elif len(dict_ref) > len(checked):
        print("there are some missing curves")
    
    print("\n\nMISSING CURVES::\n")
    #checks for the curves that should be in the input file but are not there either due to it being misspelled or because the unit is incorrect
    for x in dict_ref:
        if x not in checked:
            print(x, dict_ref[x])
    print("\n\nEXTRA CURVES::\n")
    for m in checked:
            if m not in dict_ref:
                print(m, checked[x])

All curvenames and units are correct
89  Curves are in the las file  MP PROCESS.las
